In [ ]:
## 模型建置
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input

class_num = 12

base_model = MobileNetV2(
    input_shape=(224,224,3),
    include_top=False,
    weights="imagenet",
    )

from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense

inputs = tf.keras.Input(shape=(224,224,3))
# 這邊是可以直接添加前處理，讓後續的data_input、prediction不需要前處理
x = preprocess_input(inputs)
x = base_model(x)#base_model.output # 如果要使用上面的前處理，inputs改成x
#在後面接權平均池化
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(class_num, activation='softmax')(x)

model = Model(inputs=inputs, outputs=predictions)


In [ ]:
model.summary()

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_17 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 tf.math.truediv_6 (TFOpLamb  (None, 224, 224, 3)      0         
 da)                                                             
                                                                 
 tf.math.subtract_6 (TFOpLam  (None, 224, 224, 3)      0         
 bda)                                                            
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d_9   (None, 1280)             0         
 (GlobalAveragePooling2D)                                  

In [9]:
from keras.layers.preprocessing.image_preprocessing import Rescaling
#讀取資料
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet_v2 import preprocess_input
TrainDataGenerator = ImageDataGenerator()
    # rotation_range=30,  # 随机旋转角度范围为 ±30 度
    # width_shift_range=0.2,  # 随机水平平移的范围为 ±20% 的图像宽度
    # height_shift_range=0.2,  # 随机垂直平移的范围为 ±20% 的图像高度
    # shear_range=0.2,  # 随机剪切的程度为 ±20% 的图像宽度
    # zoom_range=0.2,  # 随机缩放范围为 ±20%
    # preprocess_input=,)
traindata = TrainDataGenerator.flow_from_directory(
          directory= "/content/drive/MyDrive/AI課程/專題/colab_training/select_0827_12/train",\
          target_size=(224,224),class_mode='categorical',batch_size=32,shuffle=True)

valDataGenerator = ImageDataGenerator()
valdata = valDataGenerator.flow_from_directory(
          directory="/content/drive/MyDrive/AI課程/專題/colab_training/select_0827_12/val", \
          target_size=(224,224),class_mode='categorical',batch_size=32)

testDataGenerator = ImageDataGenerator()
testdata = testDataGenerator.flow_from_directory(
          directory="/content/drive/MyDrive/AI課程/專題/colab_training/select_0827_12/test", \
          target_size=(224,224),class_mode='categorical',batch_size=32)

Found 1087 images belonging to 12 classes.
Found 248 images belonging to 12 classes.
Found 217 images belonging to 12 classes.


In [ ]:
# 編譯模型
from tensorflow.keras.optimizers import Adam
optimizer = Adam()
learning_rate = 0.0002
optimizer.lr.assign(learning_rate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# 設定監控方法與條件
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint( filepath="/content/drive/MyDrive/AI課程/專題/colab_training/model_save/mobileV2_12.h5",
                monitor='val_accuracy',
                verbose=0,
                save_best_only=True,
                save_weights_only=False,
                mode='auto')

earlystop = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=5, verbose=1, mode='auto')

In [ ]:
# 模型訓練
history = model.fit(traindata, epochs=100, validation_data=valdata, callbacks=[earlystop, checkpoint], verbose = 1)

Epoch 1/100
34/34 [==============================] - 119s 3s/step - loss: 0.9329 - accuracy: 0.7452 - val_loss: 0.7779 - val_accuracy: 0.7339
Epoch 2/100
34/34 [==============================] - 87s 3s/step - loss: 0.0341 - accuracy: 0.9945 - val_loss: 0.4825 - val_accuracy: 0.8226
Epoch 3/100
34/34 [==============================] - 87s 3s/step - loss: 0.0071 - accuracy: 1.0000 - val_loss: 0.3331 - val_accuracy: 0.9032
Epoch 4/100
34/34 [==============================] - 88s 3s/step - loss: 0.0050 - accuracy: 0.9991 - val_loss: 0.2963 - val_accuracy: 0.9153
Epoch 5/100
34/34 [==============================] - 88s 3s/step - loss: 0.0034 - accuracy: 1.0000 - val_loss: 0.2460 - val_accuracy: 0.9234
Epoch 6/100
34/34 [==============================] - 86s 3s/step - loss: 0.0045 - accuracy: 0.9991 - val_loss: 0.2532 - val_accuracy: 0.9194
Epoch 7/100
34/34 [==============================] - 87s 3s/step - loss: 0.0028 - accuracy: 0.9991 - val_loss: 0.2202 - val_accuracy: 0.9395
Epoch 8/100


In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["acc","val_acc","loss","val_loss"])
plt.show(block=True)

In [8]:
from tensorflow.keras.models import load_model
model_load_path = "/content/drive/MyDrive/AI課程/專題/報告/tf組內報告/mobileV2_12_acc96.h5"
model_save_path = "/content/drive/MyDrive/AI課程/專題/報告/tf組內報告/thi101_mobileV2"
model = load_model(model_load_path)

In [11]:
# 測試集預測
evaluation_results = model.evaluate(testdata)
print("loss:", evaluation_results[0])
print("acc:", evaluation_results[1])

7/7 [==============================] - 73s 10s/step - loss: 0.0149 - accuracy: 0.9954
loss: 0.014901514165103436
acc: 0.9953917264938354


In [12]:
model.save(model_save_path)

In [ ]:
import cv2
import matplotlib.pyplot as plt
img_path = "1.JPG"
img = cv2.imread(img_path)
plt.figure(figsize=(4, 4))
# cv2內建顏色讀取為，BGR，訓練時顏色讀取為RGB，這邊需要做轉換
# 有些model在使用Transformer learning時，它們是用BGR做訓練，那就要讓圖片是BGR(EX. VGG16)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.axis("off")
plt.show()

In [ ]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import numpy as np
# 示範沒有前處理的輸出結果
img_pre = preprocess_input(img)
img_pre

In [ ]:
img_pre.shape

In [ ]:
img_224 = tf.image.resize(img_pre, (224, 224))

In [ ]:
img_224.shape

In [ ]:
img_224_4 = np.expand_dims(img_224,0)

In [ ]:
img_224_4.shape

In [ ]:
pred = model.predict(img_224_4)
pred

In [ ]:
index = np.argmax(pred)
index

In [ ]:
name_list = ["GM8","QI8","QY8","TS"]
result = name_list[index]
print("prediction :",result)

In [ ]:
## 熱成像，要先觀察整個模型
base_model.summary()

In [ ]:
model.summary()

In [ ]:
# 設置一個模型, 此模型會傳回最終卷積層的輸出
last_conv_layer_name = "Conv_1"  # 最後一層卷積層名稱
last_conv_layer = base_model.get_layer(last_conv_layer_name)
last_conv_layer_model = tf.keras.Model(base_model.inputs, last_conv_layer.output)

In [ ]:
# shape[1:] 主要是移除第 0 軸(批次量軸),這是因為 input()的 shape參數不可以包含批次量
classifier_input = tf.keras.Input(shape=last_conv_layer.output.shape[1:])
x = classifier_input

x = GlobalAveragePooling2D()(x)
# x = Dense(2048, activation='relu')(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(class_num, activation='softmax')(x)

classifier_model = tf.keras.Model(classifier_input, x)

In [ ]:
import tensorflow as tf
with tf.GradientTape() as tape:
    # 將影像傳入至模型運算至最後一層卷積後傳出結果
    last_conv_layer_output = last_conv_layer_model(img_224_4)
    tape.watch(last_conv_layer_output)
    # 取得最高分類類別的激活 channel(下面三行指令)
    preds = classifier_model(last_conv_layer_output)
    # 取得最高分類別的梯度
    top_pred_index = tf.argmax(preds[0])
    top_class_channel = preds[:, top_pred_index]
#　計算＂最高分的預測類別＂相對於＂最終卷積之輸出特徵圖＂的梯度
grads = tape.gradient(top_class_channel, last_conv_layer_output)

In [ ]:
# 梯度池化及channel重要性加權
# 會計算出一個向量，它的每個元素都是特定channel的平均梯度(它量化了每個channel對最高分類別的重要程度)
# 會陸續沿著第0,1,2軸做平均,因此這三軸會消失,最後只剩下第3軸(channel)
pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2)).numpy()
last_conv_layer_output = last_conv_layer_output.numpy()[0]
for i in range(pooled_grads.shape[-1]):
    last_conv_layer_output[:, :, i] *= pooled_grads[i]
heatmap = np.mean(last_conv_layer_output, axis=-1)

In [ ]:
# 將熱力圖正規化
import matplotlib.pyplot as plt
heatmap = np.maximum(heatmap, 0)
heatmap /= np.max(heatmap)
plt.matshow(heatmap)

In [ ]:
# 疊加熱力圖和原始影像
import matplotlib as mpl
# 載入原始影像
img = tf.keras.utils.load_img(img_path)
img = tf.keras.utils.img_to_array(img)
# 將熱力圖中的數值調整到 0-255 之間
heatmap = np.uint8(255 * heatmap)
# 使用 jet 色盤對熱力圖著色(以下三行)
jet = mpl.colormaps.get_cmap("jet")
jet_colors = jet(np.arange(256))[:, :3]
jet_heatmap = jet_colors[heatmap]
# 創建一個包含重新著色之熱力圖的影像(以下三行)
jet_heatmap = tf.keras.utils.array_to_img(jet_heatmap)
jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
jet_heatmap = tf.keras.utils.img_to_array(jet_heatmap)
# 疊加熱力圖與原圖, 熱力圖的不透明度為 40%
superimposed_img = jet_heatmap * 0.4 + img
superimposed_img = tf.keras.utils.array_to_img(superimposed_img)
rotated_superimposed_img = superimposed_img.rotate(-90)

save_path = "combine.jpg"
rotated_superimposed_img.save(save_path) # 儲存疊加後的影像